# Lab: Gradio Apps
* **Created by:** Eric Martinez
* **For:** CSCI 3351
* **At:** University of Texas Rio-Grande Valley

#### Gradio

Gradio is an open-source Python library that allows developers to create and prototype user interfaces (UIs) and APIs for machine learning applications and LLMs quickly and easily. Three reasons it is a great tool are:

* Simple and intuitive: Gradio makes it easy to create UIs with minimal code, allowing developers to focus on their models.
* Versatile: Gradio supports a wide range of input and output types, making it suitable for various machine learning tasks.
* Shareable: Gradio interfaces can be shared with others, enabling collaboration and easy access to your models.

## Get Setup:
Create a `.env` file in this folder. Add secrets as key-value pairs in the `.env` file

In [1]:
%%writefile .env
OPENAI_API_BASE=<my API base>
OPENAI_API_KEY=<your API key to my service>

Overwriting .env


Edit the resulting `.env` file with your `OPENAI_API_BASE` and `OPENAI_API_KEY`

#### Install Dependencies

In [1]:
!pip -q install --upgrade python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
!pip -q install --upgrade openai


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
!pip -q install --upgrade gradio


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


#### Function for interacting with GPT

In [4]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)  # take environment variables from .env.

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)

def chat_completion(
    message,
    model="gpt-4",
    prompt="You are a helpful assistant.",
    temperature=0,
    messages=[],
):
    # Add the prompt to the messages list
    if prompt is not None:
        messages = [{"role": "system", "content": prompt}] + messages

    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]

    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages

    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )

    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

# Resume Evaluator V1 (Input Textbox & Output Textbox

In [1]:
import gradio as gr
 
def resume_evaluator(resume):
    prompt = "Given a resume in text, Evaluate and rate it from 1-10 and point out what changes should be made if any on the resume. provide an example with changes that you see fit on the given resume to make it better."
    message = resume
    reply = chat_completion(message, prompt=prompt)
    return reply
    
with gr.Blocks() as app:
    with gr.Row():
        resume = gr.Textbox(label="Input Resume", lines = 15)
        output = gr.Textbox(label="Output", interactive=False, lines =15)
    with gr.Row():
        btn = gr.Button(value ="Send")
        btn.click(resume_evaluator, inputs = [resume], outputs = [output])
    app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c8f7bd7edb4b5bf451.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Resume Evaluator V2 (Input, Output and Joblisting)

In [2]:
import gradio as gr
 
def resume_evaluator(resume, job_listing):
    prompt = """Given on the resume in text and the job listing the user wants to apply to.
                Provide a rating 1-10 on how good the resume is and how much it fits the job listing.
                Provide an example of changes you would make to the resume to make it better fot the job listing.
            """
    message = f"{resume}, {joblisting}" 
    reply = chat_completion(message, prompt=prompt)
    return reply
    
with gr.Blocks() as app:
    with gr.Row():
        resume = gr.Textbox(label="Input Resume", lines = 15)
        joblisting = gr.Textbox(label="Joblisting", lines =15)
    with gr.Row():
        btn = gr.Button(value ="Send")
    results = gr.Textbox(label="Results", interactive=False, lines =15)
    btn.click(resume_evaluator, inputs = [resume, joblisting], outputs = [results])
    app.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://7804835832fc0428cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Install PDF Dependency

In [40]:
pip install PyPDF2


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Function to read PDF Rubric from files

In [3]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text

pdf_path = "/Users/kevin/Desktop/Ai Apps/HW/resume-assessment-rubric.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

 Stage 1: NoExperienceStage 2: Some ExperienceStage 3: Relevant ExperienceStage 4: ProfessionalProfessionalismResume is difficult tonavigate. Informationmay not be in the rightplace. Non-standardformat. Informationmight bemisrepresented due tolack of understandingfrom the candidate.Resume is clear and easy tonavigate, however, significantedits could make it much higherquality.Resume is clear and easy to navigate.Candidate has put significant time andeffort into editing.Resume is clear andeasy to navigate.Candidate has putsignificant time andeffort into editing andcurating the content forthis specific role.WorkExperienceCandidate has neverworked before.Candidate has worked before butpossibly not in this field.Candidate has held an internship intechnology. Better would be aninternship in this area.Candidate has existingprofessional workexperience in this area.Teamwork andProfessionalPracticesCandidate has no workexperience, teamexperience. Possiblyfamiliar with Agile butonly from an acad

# Resume Evaluator V3 (Use PDF rubric, Input, Output and Joblisting)

In [6]:
import gradio as gr
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text

pdf_path = "/Users/kevin/Desktop/Ai Apps/HW/resume-assessment-rubric.pdf"
extracted_text = extract_text_from_pdf(pdf_path)


def resume_evaluator(resume, job_listing):
    pdf_path = "/Users/kevin/Desktop/Ai Apps/HW/resume-assessment-rubric.pdf"
    extracted_text = extract_text_from_pdf(pdf_path)
    
    prompt = """Given on the resume in text use the rubric categories to rate the resume in every category by level
                Also provide on a scale of 1-10 how well the resume is given the joblisting. Is it a good fit? 
                does it have the proper key words relating to the job, these are things to take into account.
            """
    message = f"{resume}, {joblisting}, This is the Resume rubric {extracted_text}" 
    reply = chat_completion(message, prompt=prompt)
    return reply
    
with gr.Blocks() as app:
    with gr.Row():
        resume = gr.Textbox(label="Input Resume", lines = 15)
        joblisting = gr.Textbox(label="Joblisting", lines =15)
    with gr.Row():
        btn = gr.Button(value ="Send")
    results = gr.Textbox(label="Results", interactive=False, lines =15)
    btn.click(resume_evaluator, inputs = [resume, joblisting], outputs = [results])
    app.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://00234b9a3dac98ba48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Resume Evaluator V4
- User inputs PDF file for resume instead of text
- User inputs Joblisting link rather than text
- All other V3 features.

In [11]:
import gradio as gr
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text

def fetch_job_listing(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return "Error: Unable to fetch job listing from the provided URL."
    except Exception as e:
        return f"Error: {str(e)}"

pdf_path = "/Users/kevin/Desktop/Ai Apps/HW/resume-assessment-rubric.pdf"
extracted_text = extract_text_from_pdf(pdf_path)


def resume_evaluator(resume, job_listing):
    pdf_path = "/Users/kevin/Desktop/Ai Apps/HW/resume-assessment-rubric.pdf"
    extracted_text = extract_text_from_pdf(pdf_path)
    resume = extract_text_from_pdf(resume)
    job_listing = fetch_job_listing(job_listing)
    
    prompt = """Given on the resume in text use the rubric categories to rate the resume in every category by level
                Also provide on a scale of 1-10 how well the resume is given the joblisting. Is it a good fit? 
                does it have the proper key words relating to the job, these are things to take into account.
            """
    message = f"{resume}, {joblisting}, This is the Resume rubric {extracted_text}" 
    reply = chat_completion(message, prompt=prompt)
    return reply
    
with gr.Blocks() as app:
    with gr.Row():
        resume = gr.File(label="Drop Resume")
        joblisting = gr.Textbox(label="Joblisting URL", lines =10)
    with gr.Row():
        btn = gr.Button(value ="Send")
    results = gr.Textbox(label="Results", interactive=False, lines =15)
    btn.click(resume_evaluator, inputs = [resume, joblisting], outputs = [results])
    app.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://ff4e1ae301e6347359.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
